In [1]:
%matplotlib inline
from misc import *
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
seed = 8
data_dir = 'data'
train_size = 94*3*7
batch_size = 64
n_lstm = 100
epochs = 1000
noise = 0.1

In [3]:
X_train, X_validation, X_test, Y_train, Y_validation, Y_test, le = load_prepare_data(data_dir, train_size, seed)
model, lstm_activations = prepare_model(100, 'accuracy', X_train, Y_train, (0.0, 0.0), (0.0, 0.0))

/home/husein/space/text-dataset/lstm-visualization/misc.py:83: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  model = Model(input=input_layer, output=output_layer)


In [ ]:
model, history = fit_model(model, X_train, Y_train, X_validation, Y_validation, seed, epochs=100, patience=15, batch_size=64, verbosity=1)

/home/husein/space/text-dataset/lstm-visualization/misc.py:102: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  verbose=verbosity)


Train on 1974 samples, validate on 277 samples
Epoch 1/100
1974/1974 [==============================] - 2s - loss: 4.5528 - acc: 0.0081 - val_loss: 4.5458 - val_acc: 0.0072
Epoch 2/100
1974/1974 [==============================] - 2s - loss: 4.5446 - acc: 0.0101 - val_loss: 4.5446 - val_acc: 0.0072
Epoch 3/100
1974/1974 [==============================] - 2s - loss: 4.5321 - acc: 0.0177 - val_loss: 4.4976 - val_acc: 0.0289
Epoch 4/100
1974/1974 [==============================] - 2s - loss: 4.4323 - acc: 0.0172 - val_loss: 4.4545 - val_acc: 0.0289
Epoch 5/100
1974/1974 [==============================] - 2s - loss: 4.4078 - acc: 0.0167 - val_loss: 4.5390 - val_acc: 0.0217
Epoch 6/100
1974/1974 [==============================] - 2s - loss: 4.4490 - acc: 0.0208 - val_loss: 4.4317 - val_acc: 0.0144
Epoch 7/100
1974/1974 [==============================] - 2s - loss: 4.2946 - acc: 0.0218 - val_loss: 4.2269 - val_acc: 0.0253
Epoch 8/100
1974/1974 [==============================] - 2s - loss: 4.2

1974/1974 [==============================] - 2s - loss: 1.0627 - acc: 0.6662 - val_loss: 1.1609 - val_acc: 0.6101
Epoch 66/100
1974/1974 [==============================] - 2s - loss: 0.9936 - acc: 0.6793 - val_loss: 1.1411 - val_acc: 0.6354
Epoch 67/100
1974/1974 [==============================] - 2s - loss: 1.0325 - acc: 0.6707 - val_loss: 1.2369 - val_acc: 0.5921
Epoch 68/100
1974/1974 [==============================] - 2s - loss: 0.9647 - acc: 0.7011 - val_loss: 1.0735 - val_acc: 0.6462
Epoch 69/100
1974/1974 [==============================] - 2s - loss: 0.9660 - acc: 0.6879 - val_loss: 1.0804 - val_acc: 0.6462
Epoch 70/100
1974/1974 [==============================] - 2s - loss: 0.9037 - acc: 0.7112 - val_loss: 1.0106 - val_acc: 0.6606
Epoch 71/100
1974/1974 [==============================] - 2s - loss: 0.8354 - acc: 0.7234 - val_loss: 1.0039 - val_acc: 0.7004
Epoch 72/100
1974/1974 [==============================] - 2s - loss: 0.8372 - acc: 0.7310 - val_loss: 0.9235 - val_acc: 0.69

In [ ]:
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

def custom_activation(x):
    return x

act = lstm_activations.predict(X_train)
labels = le.inverse_transform(np.argmax(Y_train, axis=1))
X_autoencoder, Y_autoencoder = prepare_autoencoder_input(act, noise, seed, min_=10, max_=65)

decoded3D, encoded3D = prepare_autoencoder(n_lstm, 3, dense_1=600, dense_2=150, activation=custom_activation)
decoded3D, _ = fit_autoencoder(decoded3D, X_autoencoder, Y_autoencoder, seed, patience=5, verbosity=1,epochs=5)

/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Train on 86856 samples, validate on 21714 samples
Epoch 1/5
86856/86856 [==============================] - 1s - loss: 0.0384 - val_loss: 0.0259
Epoch 2/5
 5376/86856 [>.............................] - ETA: 1s - loss: 0.0257

In [ ]:
act = lstm_activations.predict(X_train)
labels = le.inverse_transform(np.argmax(Y_train, axis=1))
set_labels = sorted(set(labels))
plot_3D(set_labels, encoded3D, act, labels, 'LSTM logits 3D', x_l=-2.5,
            x_h=2.5, y_l=-2.5, y_h=2.5, figsize=(13, 13), pointsize=5, max_steps=90, log=True)